###  ---------------------------------------------------------------------------------------------------------------------------------------
##### Copyright (c) Rajdeep Biswas
##### Licensed under the MIT license.
###### File: get_stock_data.ipynb
###### Date: 09/12/2021
###  ---------------------------------------------------------------------------------------------------------------------------------------

### Table of Contents

* [Initial Configurations](#IC)
    * [Import Libraries](#IL)
    * [Autheticate the AML Workspace](#AML)
* [Transform Data (Silver)](#TSD)
    * [Setup Directory Structure](#SD)
    * [Merge Daily Stock Data](#DD)
    * [Merge Historical Stock Dividend Data](#HD)
    * [Merge Historical Stock Split Data](#SS)
    * [Copy Stock Financial Data](#SFD)      

### Initial Configurations <a class="anchor" id="IC"></a>

#### Import Libraries <a class="anchor" id="IL"></a>

In [1]:
#Import required Libraries
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

#pip install pandas_datareader
import pandas_datareader.data as web
import pandas as pd
import datetime as dt

import azureml.core
import azureml.automl
from azureml.core import Workspace, Dataset, Datastore

#### Autheticate the AML Workspace <a class="anchor" id="AML"></a>

In [3]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.33.0 to work with houston-techsummit-workspace


### Transform Data (Silver) <a class="anchor" id="TSD"></a>
- Raw data extraction for the file, API based and web datasets. Let us call this __Bronze Layer__.
- Data transformation using python from Raw to Processed stage. We will call this __Silver Layer__.
- Finally store the processed data using standard taxonomy in a SQL based serving layer. We will call this __Gold Layer__.

#### Setup Directory Structure <a class="anchor" id="SD"></a>

In [5]:
data_folder = os.path.join(os.getcwd(), 'data')

#Create the data directory
os.makedirs(data_folder, exist_ok=True)

#Create the bronze, silver and gold folders
bronze_data_folder = data_folder +"/bronze"
os.makedirs(bronze_data_folder, exist_ok=True)

silver_data_folder = data_folder +"/silver"
os.makedirs(silver_data_folder, exist_ok=True)

gold_data_folder = data_folder +"/gold"
os.makedirs(gold_data_folder, exist_ok=True)

#Create sub folder for stock daily close data
file_data_bronze = bronze_data_folder +"/snp500"
os.makedirs(file_data_bronze, exist_ok=True)

#Create sub folder for dividends
dividend_data_bronze = bronze_data_folder +"/snp500_dividends"
os.makedirs(dividend_data_bronze, exist_ok=True)

#Create sub folder for splits
split_data_bronze = bronze_data_folder +"/snp500_splits"
os.makedirs(split_data_bronze, exist_ok=True)

#Create sub folder for sectors
sector_data_bronze = bronze_data_folder +"/snp500_sectors"
os.makedirs(sector_data_bronze, exist_ok=True)

#Create sub folder for financials
financial_data_bronze = bronze_data_folder +"/snp500_financial"
os.makedirs(financial_data_bronze, exist_ok=True)

#### Merge Daily Stock Data <a class="anchor" id="DD"></a>

In [26]:
import glob
all_daily_files_bronze = glob.glob(file_data_bronze + "/*.csv")

In [27]:
list_content = []

for filename in all_daily_files_bronze:
    temp_df = pd.read_csv(filename, index_col=None, header=0)
    #Extract the stockname from the filename and add it
    stock_name = os.path.basename(filename).split('_')[0]
    temp_df['Symbol'] = stock_name
    list_content.append(temp_df)

In [28]:
df_daily_stock_bronze = pd.concat(list_content, axis=0, ignore_index=True)

In [29]:
df_daily_stock_bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624555 entries, 0 to 624554
Data columns (total 8 columns):
Date         624555 non-null object
High         624555 non-null float64
Low          624555 non-null float64
Open         624555 non-null float64
Close        624555 non-null float64
Volume       624555 non-null float64
Adj Close    624555 non-null float64
Symbol       624555 non-null object
dtypes: float64(6), object(2)
memory usage: 38.1+ MB


In [30]:
df_daily_stock_bronze.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Symbol
0,2016-09-01,36.709999,35.880001,36.529999,36.240002,6940600.0,34.931732,AAL
1,2016-09-02,36.700001,36.000000,36.000000,36.529999,5429100.0,35.211254,AAL
2,2016-09-06,36.970001,36.360001,36.450001,36.959999,5501700.0,35.625729,AAL
3,2016-09-07,39.520000,37.419998,37.500000,38.750000,13775000.0,37.351112,AAL
4,2016-09-08,39.410000,38.430000,38.680000,39.349998,7101500.0,37.929455,AAL


In [31]:
#Let us quickly handle null values
df_daily_stock_bronze.isnull().sum()

Date         0
High         0
Low          0
Open         0
Close        0
Volume       0
Adj Close    0
Symbol       0
dtype: int64

Seems like there is no null values to handle. Cool!!!

In [35]:
#Write in silver directory
output_file_name = silver_data_folder + '/snp500_daily_ticker.csv'
df_daily_stock_bronze.to_csv(output_file_name, index=False)    

In [36]:
df_snp500_daily_ticker = pd.read_csv(output_file_name)
df_snp500_daily_ticker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624555 entries, 0 to 624554
Data columns (total 8 columns):
Date         624555 non-null object
High         624555 non-null float64
Low          624555 non-null float64
Open         624555 non-null float64
Close        624555 non-null float64
Volume       624555 non-null float64
Adj Close    624555 non-null float64
Symbol       624555 non-null object
dtypes: float64(6), object(2)
memory usage: 38.1+ MB


In [37]:
df_snp500_daily_ticker.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Symbol
0,2016-09-01,36.709999,35.880001,36.529999,36.240002,6940600.0,34.931732,AAL
1,2016-09-02,36.700001,36.000000,36.000000,36.529999,5429100.0,35.211254,AAL
2,2016-09-06,36.970001,36.360001,36.450001,36.959999,5501700.0,35.625729,AAL
3,2016-09-07,39.520000,37.419998,37.500000,38.750000,13775000.0,37.351112,AAL
4,2016-09-08,39.410000,38.430000,38.680000,39.349998,7101500.0,37.929455,AAL


#### Merge Historical Stock Dividend Data <a class="anchor" id="HD"></a>

In [38]:
all_dividend_files_bronze = glob.glob(dividend_data_bronze + "/*.csv")

In [39]:
list_content = []

for filename in all_dividend_files_bronze:
    temp_df = pd.read_csv(filename, index_col=None, header=0)
    #Extract the stockname from the filename and add it
    stock_name = os.path.basename(filename).split('_')[0]
    temp_df['Symbol'] = stock_name
    list_content.append(temp_df)

In [40]:
df_dividend_stock_bronze = pd.concat(list_content, axis=0, ignore_index=True)

In [43]:
df_dividend_stock_bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7772 entries, 0 to 7771
Data columns (total 4 columns):
Unnamed: 0    7772 non-null object
action        7772 non-null object
value         7772 non-null float64
Symbol        7772 non-null object
dtypes: float64(1), object(3)
memory usage: 243.0+ KB


In [44]:
df_dividend_stock_bronze.head()

,Unnamed: 0,action,value,Symbol
0,2020-02-04,DIVIDEND,0.1,AAL
1,2019-11-05,DIVIDEND,0.1,AAL
2,2019-08-06,DIVIDEND,0.1,AAL
3,2019-05-07,DIVIDEND,0.1,AAL
4,2019-02-05,DIVIDEND,0.1,AAL


In [47]:
df_dividend_stock_bronze.rename(columns={'Unnamed: 0' :'Date'}, inplace=True )
df_dividend_stock_bronze.head()

,Date,action,value,Symbol
0,2020-02-04,DIVIDEND,0.1,AAL
1,2019-11-05,DIVIDEND,0.1,AAL
2,2019-08-06,DIVIDEND,0.1,AAL
3,2019-05-07,DIVIDEND,0.1,AAL
4,2019-02-05,DIVIDEND,0.1,AAL


In [48]:
df_dividend_stock_bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7772 entries, 0 to 7771
Data columns (total 4 columns):
Date      7772 non-null object
action    7772 non-null object
value     7772 non-null float64
Symbol    7772 non-null object
dtypes: float64(1), object(3)
memory usage: 243.0+ KB


In [51]:
#Let us quickly handle null values
df_dividend_stock_bronze.isnull().sum()

Date      0
action    0
value     0
Symbol    0
dtype: int64

Seems like there is no null values to handle. Cool!!!

In [52]:
#Write in silver directory
output_file_name = silver_data_folder + '/snp500_dividends.csv'
df_dividend_stock_bronze.to_csv(output_file_name, index=False)    

In [53]:
df_snp500_dividend = pd.read_csv(output_file_name)
df_snp500_dividend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7772 entries, 0 to 7771
Data columns (total 4 columns):
Date      7772 non-null object
action    7772 non-null object
value     7772 non-null float64
Symbol    7772 non-null object
dtypes: float64(1), object(3)
memory usage: 243.0+ KB


In [54]:
df_snp500_dividend.head()

,Date,action,value,Symbol
0,2020-02-04,DIVIDEND,0.1,AAL
1,2019-11-05,DIVIDEND,0.1,AAL
2,2019-08-06,DIVIDEND,0.1,AAL
3,2019-05-07,DIVIDEND,0.1,AAL
4,2019-02-05,DIVIDEND,0.1,AAL


In [66]:
df_snp500_dividend['action'].value_counts()

DIVIDEND    7772
Name: action, dtype: int64

#### Merge Historical Stock Split Data <a class="anchor" id="SS"></a>

In [55]:
all_split_files_bronze = glob.glob(split_data_bronze + "/*.csv")

In [56]:
list_content = []

for filename in all_split_files_bronze:
    temp_df = pd.read_csv(filename, index_col=None, header=0)
    #Extract the stockname from the filename and add it
    stock_name = os.path.basename(filename).split('_')[0]
    temp_df['Symbol'] = stock_name
    list_content.append(temp_df)

In [57]:
df_split_stock_bronze = pd.concat(list_content, axis=0, ignore_index=True)

In [58]:
df_split_stock_bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7870 entries, 0 to 7869
Data columns (total 4 columns):
Unnamed: 0    7870 non-null object
action        7870 non-null object
value         7870 non-null float64
Symbol        7870 non-null object
dtypes: float64(1), object(3)
memory usage: 246.1+ KB


In [59]:
df_split_stock_bronze.head()

,Unnamed: 0,action,value,Symbol
0,2020-02-04,DIVIDEND,0.1,AAL
1,2019-11-05,DIVIDEND,0.1,AAL
2,2019-08-06,DIVIDEND,0.1,AAL
3,2019-05-07,DIVIDEND,0.1,AAL
4,2019-02-05,DIVIDEND,0.1,AAL


In [60]:
df_split_stock_bronze.rename(columns={'Unnamed: 0' :'Date'}, inplace=True )
df_split_stock_bronze.head()

,Date,action,value,Symbol
0,2020-02-04,DIVIDEND,0.1,AAL
1,2019-11-05,DIVIDEND,0.1,AAL
2,2019-08-06,DIVIDEND,0.1,AAL
3,2019-05-07,DIVIDEND,0.1,AAL
4,2019-02-05,DIVIDEND,0.1,AAL


In [61]:
#Let us quickly handle null values
df_split_stock_bronze.isnull().sum()

Date      0
action    0
value     0
Symbol    0
dtype: int64

Seems like there is no null values to handle. Cool!!!

In [62]:
#Write in silver directory
output_file_name = silver_data_folder + '/snp500_dividend_split.csv'
df_split_stock_bronze.to_csv(output_file_name, index=False)    

In [63]:
df_snp500_dividend_split = pd.read_csv(output_file_name)
df_snp500_dividend_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7870 entries, 0 to 7869
Data columns (total 4 columns):
Date      7870 non-null object
action    7870 non-null object
value     7870 non-null float64
Symbol    7870 non-null object
dtypes: float64(1), object(3)
memory usage: 246.1+ KB


In [64]:
df_snp500_dividend_split.head()

,Date,action,value,Symbol
0,2020-02-04,DIVIDEND,0.1,AAL
1,2019-11-05,DIVIDEND,0.1,AAL
2,2019-08-06,DIVIDEND,0.1,AAL
3,2019-05-07,DIVIDEND,0.1,AAL
4,2019-02-05,DIVIDEND,0.1,AAL


In [65]:
df_snp500_dividend_split['action'].value_counts()

DIVIDEND    7804
SPLIT         66
Name: action, dtype: int64

#### Copy Stock Financial Data <a class="anchor" id="SFD"></a>

The financial data is a superset of the sector data hence we do not need to use the sector data.

In [82]:
import shutil, os
snp500_financial_file = financial_data_bronze + '/snp500_financial.csv'
shutil.copy(snp500_financial_file, silver_data_folder)

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-cpu-ds12-v2/code/Users/rabiswas/StockPrediction/data/silver/snp500_financial.csv'

In [83]:
output_file_name = silver_data_folder + '/snp500_financial.csv'
df_snp500_financial = pd.read_csv(output_file_name)
df_snp500_financial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 15 columns):
Unnamed: 0        505 non-null int64
Symbol            505 non-null object
Name              505 non-null object
Sector            505 non-null object
Price             505 non-null float64
Price/Earnings    503 non-null float64
Dividend Yield    505 non-null float64
Earnings/Share    505 non-null float64
52 Week Low       505 non-null float64
52 Week High      505 non-null float64
Market Cap        505 non-null int64
EBITDA            505 non-null float64
Price/Sales       505 non-null float64
Price/Book        497 non-null float64
SEC Filings       505 non-null object
dtypes: float64(9), int64(2), object(4)
memory usage: 59.3+ KB


In [78]:
df_snp500_financial.head()

,Unnamed: 0,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,138721055226,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,10783419933,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,102121042306,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,181386347059,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,98765855553,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [84]:
df_snp500_financial.drop(['Unnamed: 0'], axis=1, inplace = True)
df_snp500_financial.to_csv(output_file_name, index=False)    

In [85]:
output_file_name = silver_data_folder + '/snp500_financial.csv'
df_snp500_financial = pd.read_csv(output_file_name)
df_snp500_financial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 14 columns):
Symbol            505 non-null object
Name              505 non-null object
Sector            505 non-null object
Price             505 non-null float64
Price/Earnings    503 non-null float64
Dividend Yield    505 non-null float64
Earnings/Share    505 non-null float64
52 Week Low       505 non-null float64
52 Week High      505 non-null float64
Market Cap        505 non-null int64
EBITDA            505 non-null float64
Price/Sales       505 non-null float64
Price/Book        497 non-null float64
SEC Filings       505 non-null object
dtypes: float64(9), int64(1), object(4)
memory usage: 55.4+ KB


In [86]:
df_snp500_financial.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,138721055226,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,10783419933,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,102121042306,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,181386347059,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,98765855553,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [92]:
#It is easier to view the data if we transpose
df_snp500_financial.head(3).transpose()

,0,1,2
Symbol,MMM,AOS,ABT
Name,3M Company,A.O. Smith Corp,Abbott Laboratories
Sector,Industrials,Industrials,Health Care
Price,222.89,60.24,56.27
Price/Earnings,24.31,27.76,22.51
Dividend Yield,2.33286,1.14796,1.90898
Earnings/Share,7.92,1.7,0.26
52 Week Low,259.77,68.39,64.6
52 Week High,175.49,48.925,42.28
Market Cap,138721055226,10783419933,102121042306


In [88]:
#!pip install yfinance

import yfinance as yf

##### The S&P 500 financial data captured was not worth except for the sector information since there is no date component in the data:

In [94]:
#!pip install yahoo_fin
import yahoo_fin.stock_info as si

In [95]:
quote = si.get_quote_table("aapl")
quote

{'1y Target Est': 167.13,
 '52 Week Range': '103.10 - 157.26',
 'Ask': '148.68 x 1100',
 'Avg. Volume': 78489468.0,
 'Beta (5Y Monthly)': 1.2,
 'Bid': '148.67 x 2200',
 "Day's Range": '148.70 - 155.48',
 'EPS (TTM)': 5.11,
 'Earnings Date': 'Oct 27, 2021 - Nov 01, 2021',
 'Ex-Dividend Date': 'Aug 06, 2021',
 'Forward Dividend & Yield': '0.88 (0.59%)',
 'Market Cap': '2.463T',
 'Open': 155.0,
 'PE Ratio (TTM)': 29.16,
 'Previous Close': 154.07,
 'Quote Price': 148.97000122070312,
 'Volume': 140893235.0}